# Trial of different APIs for interacting with Sentinel data
1. <a href="#earth-engine">Google Earth Engine</a>
2. <a href="#sentinel-sat">SentinelSat</a>
3. Snappy
4. SatPy 

<a name="earth-engine"></a>
## Earth Engine
Authentication all good - trouble using api in python as opposed to the easy javascript console

In [4]:
import ee
try:
  ee.Initialize()
  print('The Earth Engine package initialized successfully!')
except ee.EEException as e:
  print('The Earth Engine package failed to initialize! - Call $ earthengine authenticate, at the command line')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

The Earth Engine package initialized successfully!


In [9]:
import ee
import geehydro
import folium
# Get a composite of all Sentinal 2 images within a date range that include my point of interest.
poi = ee.Geometry.Point([29, 41])
image = ee.ImageCollection('COPERNICUS/S2').filterBounds(poi).filterDate('2020-12-01', '2020-12-15').min()
Map = folium.Map(location=[41, 29], zoom_start=10)
# To see a google satellite view as a basemap
Map.setOptions('HYBRID')
nir = image.select('B8')
red = image.select('B4')
ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI Layer')
ndviParams = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
Map.addLayer(ndvi, ndviParams, 'NDVI image')
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map

In [10]:
import tensorflow as tf

In [4]:
# This has details about the Earth Engine Python Authenticator client.
from ee import oauth
from google_auth_oauthlib.flow import Flow
import json

# Build the `client_secrets.json` file by borrowing the
# Earth Engine python authenticator.
client_secrets = {
    'web': {
        'client_id': oauth.CLIENT_ID,
        'client_secret': oauth.CLIENT_SECRET,
        'redirect_uris': [oauth.REDIRECT_URI],
        'auth_uri': 'https://accounts.google.com/o/oauth2/auth',
        'token_uri': 'https://accounts.google.com/o/oauth2/token'
    }
}

# Write to a json file.
client_secrets_file = 'client_secrets.json'
with open(client_secrets_file, 'w') as f:
  json.dump(client_secrets, f, indent=2)

# Start the flow using the client_secrets.json file.
flow = Flow.from_client_secrets_file(client_secrets_file,
    scopes=oauth.SCOPES,
    redirect_uri=oauth.REDIRECT_URI)

# Get the authorization URL from the flow.
auth_url, _ = flow.authorization_url(prompt='consent')

# Print instructions to go to the authorization URL.
oauth._display_auth_instructions_with_print(auth_url)
print('\n')

# The user will get an authorization code.
# This code is used to get the access token.
code = input('Enter the authorization code: \n')
flow.fetch_token(code=code)

# Get an authorized session from the flow.
session = flow.authorized_session()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&state=hHVih1bcA9VQmJVz3dZo3ERZ5k2lUp&prompt=consent&access_type=offline

The authorization workflow will generate a code, which you should paste in the box below. 




<a name="sentinel-sat"></a>
## SentinelSat
* Ok for a download but is less supported than Earth Engine, can probably give a miss

In [6]:
#%pip install sentinelsat
from sentinelsat import SentinelAPI

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt

user = 'lshc3' 
password = 'cq3FkHPiBiUbBfP' 
api = SentinelAPI('user', 'password')
footprint = geojson_to_wkt(read_geojson('C:/Users/lukec/OneDrive - University of Cambridge/Projects/GTC/data/UnitedKingdom_Bound.json'))
products = api.query(footprint,
                     producttype='SLC',
                     orbitdirection='ASCENDING')
api.download_all(products)

KeyError: 'features'

## ESA's snappy
* A pain to set-up snappy-configuration before product reading works

In [8]:
%pip install snappy
from snappy import ProductIO
import numpy as np
import matplotlib.pyplot as plt
 
p = ProductIO.readProduct('snappy/testdata/MER_FRS_L1B_SUBSET.dim')
rad13 = p.getBand('radiance_13')
w = rad13.getRasterWidth()
h = rad13.getRasterHeight()
rad13_data = np.zeros(w * h, np.float32)
rad13.readPixels(0, 0, w, h, rad13_data)
p.dispose()
rad13_data.shape = h, w
imgplot = plt.imshow(rad13_data)
imgplot.write_png('radiance_13.png')

Note: you may need to restart the kernel to use updated packages.


ImportError: cannot import name 'ProductIO' from 'snappy' (C:\ProgramData\Anaconda3\envs\sentinel\lib\site-packages\snappy\__init__.py)

## SatPy
* Not smooth sailing for downloading from ESA but could be good for combining datasets

In [10]:
%pip install satpy

  Using cached toolz-0.11.1-py3-none-any.whl (55 kB)
  Created wheel for satpy: filename=satpy-0.24.0-py2.py3-none-any.whl size=981064 sha256=f8a58963b47684d2218c3cb637b934bf13f9da08400b14c0c751a95003f446a7
  Stored in directory: c:\users\lukec\appdata\local\pip\cache\wheels\79\17\68\60f4be669c139a3c9c53e4c6b954d1ba5848860876371cb654
  Created wheel for configobj: filename=configobj-5.0.6-py3-none-any.whl size=34547 sha256=b139f66acedaf5e5fa89400d532cbd298aee3c545bd52d9e9710748c0019b912
  Stored in directory: c:\users\lukec\appdata\local\pip\cache\wheels\0d\c4\19\13d74440f2a571841db6b6e0a273694327498884dafb9cf978
  Created wheel for asciitree: filename=asciitree-0.3.3-py3-none-any.whl size=5036 sha256=47347eca83bbef3839f9b78ebfde7f895a98270b49a1a7531907fa152614bea6
  Stored in directory: c:\users\lukec\appdata\local\pip\cache\wheels\12\1c\38\0def51e15add93bff3f4bf9c248b94db0839b980b8535e72a0
Successfully built satpy configobj asciitree
Note: you may need to restart the kernel to use up

In [12]:
from datetime import datetime

from satpy import Scene, find_files_and_readers

files = find_files_and_readers(sensor='olci',
                               start_time=datetime(2018, 8, 29, 8, 26),
                               end_time=datetime(2018, 8, 29, 8, 27),
                               base_dir="/data/temp/Martin.Raspaud/s3",
                               reader='nc_olci_l1b')

scn = Scene(filenames=files)

ValueError: No reader named: nc_olci_l1b